In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('../Q1/男胎检测数据_预处理后.csv')
data

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,Y染色体浓度,X染色体浓度,13号染色体的GC含量,18号染色体的GC含量,21号染色体的GC含量,被过滤掉读段数的比例,染色体的非整倍体,怀孕次数,生产次数,胎儿是否健康
0,1,A001,31,160.0,72.0,2023-02-01,自然受孕,2023-04-29,1,11.86,...,0.025936,0.038061,0.377069,0.389803,0.399399,0.027484,NaN,1,0,是
1,2,A001,31,160.0,73.0,2023-02-01,自然受孕,2023-05-31,2,15.86,...,0.034887,0.059572,0.371542,0.384771,0.391706,0.019617,NaN,1,0,是
2,3,A001,31,160.0,73.0,2023-02-01,自然受孕,2023-06-25,3,20.14,...,0.066171,0.075995,0.377449,0.390582,0.399480,0.022312,NaN,1,0,是
3,4,A001,31,160.0,74.0,2023-02-01,自然受孕,2023-07-16,4,22.86,...,0.061192,0.052305,0.375613,0.389251,0.397212,0.023280,NaN,1,0,是
4,5,A002,32,149.0,74.0,2023-11-09,自然受孕,2024-02-19,1,13.86,...,0.059230,0.059708,0.380260,0.393618,0.404868,0.024212,NaN,2,1,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,1078,A266,30,159.0,83.4,NaN,自然受孕,2023-05-02,4,17.71,...,0.099052,0.056686,0.376861,0.389914,0.397090,0.017951,T18,1,0,是
1078,1079,A267,28,155.0,73.8,NaN,自然受孕,2023-05-17,1,11.57,...,0.098706,0.023663,0.377597,0.387901,0.404293,0.022549,T21,1,0,是
1079,1080,A267,28,155.0,74.1,NaN,自然受孕,2023-05-24,2,12.57,...,0.102088,0.080264,0.379041,0.391748,0.400433,0.021330,NaN,1,0,是
1080,1081,A267,28,155.0,74.7,NaN,自然受孕,2023-05-31,3,13.57,...,0.109855,0.074050,0.379107,0.388544,0.401030,0.022013,NaN,1,0,是


# 筛选与字段规范

In [4]:
data=data[['孕妇代码','检测孕周','孕妇BMI','Y染色体浓度']]
data

,孕妇代码,检测孕周,孕妇BMI,Y染色体浓度
0,A001,11.86,28.125000,0.025936
1,A001,15.86,28.515625,0.034887
2,A001,20.14,28.515625,0.066171
3,A001,22.86,28.906250,0.061192
4,A002,13.86,33.331832,0.059230
...,...,...,...,...
1077,A266,17.71,32.969881,0.099052
1078,A267,11.57,30.703133,0.098706
1079,A267,12.57,30.825814,0.102088
1080,A267,13.57,31.107551,0.109855


In [6]:
data=data[(data['检测孕周']>=10) & (data['检测孕周']<26)]
data.describe()

,检测孕周,孕妇BMI,Y染色体浓度
count,1073.000000,1073.000000,1073.000000
mean,16.761808,32.283321,0.077089
std,3.987793,2.972165,0.033607
min,11.000000,20.703125,0.010004
25%,13.290000,30.185195,0.051152
50%,16.000000,31.826510,0.074872
75%,20.000000,33.932236,0.098862
max,25.710000,46.875000,0.234218


## 首次达标区间构造

In [7]:
# 阈值
thr = 0.04

# 清洗与排序
data = data.copy()
data["检测孕周"] = pd.to_numeric(data["检测孕周"], errors="coerce")
data["Y染色体浓度"] = pd.to_numeric(data["Y染色体浓度"], errors="coerce")
data = data.dropna(subset=["检测孕周", "Y染色体浓度"])
data = data.sort_values(["孕妇代码", "检测孕周"], ascending=[True, True])


# 分组构造首次达标区间
def first_hit_interval(g: pd.DataFrame) -> pd.Series:
    bmi = g["孕妇BMI"].dropna().iloc[0] if g["孕妇BMI"].notna().any() else np.nan
    # convert to plain numpy float arrays to avoid ExtensionArray comparison issues
    y = g["Y染色体浓度"].to_numpy(dtype=float)
    weeks = g["检测孕周"].to_numpy(dtype=float)

    hit_idx = np.where(y >= thr)[0]
    if hit_idx.size == 0:
        # 右删失：T* > C
        C = weeks[-1]
        L, R, censor = C, np.inf, "right"
    else:
        r_pos = int(hit_idx[0])
        R = weeks[r_pos]
        if r_pos == 0:
            # 左删失：(-inf, R]
            L, censor = -np.inf, "left"
        else:
            # 区间删失：(L, R]
            L, censor = weeks[r_pos - 1], "interval"

    return pd.Series(
        {"id": g["孕妇代码"].iloc[0], "BMI": bmi, "L": L, "R": R, "censor_type": censor}
    )


intervals = (
    data.groupby("孕妇代码", group_keys=False)
    .apply(first_hit_interval)
    .reset_index(drop=True)
)

intervals

C:\Users\HP\AppData\Local\Temp\ipykernel_49024\706680993.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(first_hit_interval)


,id,BMI,L,R,censor_type
0,A001,28.125000,15.86,20.14,interval
1,A002,33.331832,-inf,13.86,left
2,A003,30.742188,-inf,13.00,left
3,A004,28.641243,-inf,11.00,left
4,A005,29.955556,-inf,12.29,left
...,...,...,...,...,...
262,A263,29.346687,-inf,12.57,left
263,A264,32.471062,-inf,11.71,left
264,A265,33.719678,-inf,11.71,left
265,A266,32.135095,-inf,13.71,left


# k均值聚类

## 特征构造

In [9]:
# 特征构造（每人一条）

# 右删失下界代理的增量（周）
delta_week = 1.0

feat = intervals.copy()

# 删失标记：right=1，其余=0
feat["censored"] = (feat["censor_type"] == "right").astype(int)

# 达标时间代理：
# - 区间/左删失：用 R（首次达标周）
# - 右删失：用 C+Δ，这里 C=L（上一步构造里 right 情况把 L 设为最后检测周）
feat["t_proxy"] = np.where(feat["censored"] == 1, feat["L"] + delta_week, feat["R"])


# 简单 z-score 标准化（避免 std=0 的数值问题）
def zscore(s: pd.Series) -> pd.Series:
    mu = s.mean()
    sigma = s.std(ddof=0)
    sigma = 1.0 if (not np.isfinite(sigma) or sigma == 0) else sigma
    return (s - mu) / sigma


feat["x1"] = zscore(feat["BMI"])
feat["x2"] = zscore(feat["t_proxy"])

# 右删失样本较低权重
feat["sample_weight"] = np.where(feat["censored"] == 1, 0.5, 1.0)

# 输出特征表（保留原值便于核查）
features = feat[["id", "x1", "x2", "censored", "sample_weight", "BMI", "t_proxy"]]
features

,id,x1,x2,censored,sample_weight,BMI,t_proxy
0,A001,-1.246572,2.205301,0,1.0,28.125000,20.14
1,A002,0.497095,0.030966,0,1.0,33.331832,13.86
2,A003,-0.370127,-0.266793,0,1.0,30.742188,13.00
3,A004,-1.073692,-0.959256,0,1.0,28.641243,11.00
4,A005,-0.633554,-0.512617,0,1.0,29.955556,12.29
...,...,...,...,...,...,...,...
262,A263,-0.837453,-0.415672,0,1.0,29.346687,12.57
263,A264,0.208840,-0.713432,0,1.0,32.471062,11.71
264,A265,0.626977,-0.713432,0,1.0,33.719678,11.71
265,A266,0.096331,-0.020968,0,1.0,32.135095,13.71


## 聚类

In [15]:
from sklearn.cluster import KMeans

k = 5  # 自行设定
X = features[["x1", "x2"]].to_numpy()
w = features["sample_weight"].to_numpy()

km = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = km.fit_predict(X, sample_weight=w)

features_km = features.copy()
features_km["cluster"] = labels
features_km

d:\Develop\miniconda\envs\mcm\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


,id,x1,x2,censored,sample_weight,BMI,t_proxy,cluster
0,A001,-1.246572,2.205301,0,1.0,28.125000,20.14,3
1,A002,0.497095,0.030966,0,1.0,33.331832,13.86,4
2,A003,-0.370127,-0.266793,0,1.0,30.742188,13.00,0
3,A004,-1.073692,-0.959256,0,1.0,28.641243,11.00,0
4,A005,-0.633554,-0.512617,0,1.0,29.955556,12.29,0
...,...,...,...,...,...,...,...,...
262,A263,-0.837453,-0.415672,0,1.0,29.346687,12.57,0
263,A264,0.208840,-0.713432,0,1.0,32.471062,11.71,4
264,A265,0.626977,-0.713432,0,1.0,33.719678,11.71,4
265,A266,0.096331,-0.020968,0,1.0,32.135095,13.71,4


In [16]:
# 按每簇 BMI 中位数对簇排序，并映射 BMI 区间

# 计算每簇 BMI 统计量
cluster_stats = features_km.groupby("cluster", as_index=False).agg(
    bmi_median=("BMI", "median"),
    bmi_min=("BMI", "min"),
    bmi_max=("BMI", "max"),
    bmi_q05=("BMI", lambda s: s.quantile(0.05)),
    bmi_q95=("BMI", lambda s: s.quantile(0.95)),
    n=("BMI", "size"),
)

# 按 BMI 中位数升序排簇，并重映射新的连续簇标签 0..k-1
cluster_stats = cluster_stats.sort_values("bmi_median", ascending=True).reset_index(
    drop=True
)
cluster_stats["cluster_sorted"] = np.arange(len(cluster_stats))

# 构造旧->新簇标签的映射并应用
label_map = dict(zip(cluster_stats["cluster"], cluster_stats["cluster_sorted"]))
features_km["cluster_sorted"] = features_km["cluster"].map(label_map)

# 为展示方便，增加区间字符串（5–95 分位优先，避免极端点）
cluster_stats["bmi_range_q5_q95"] = cluster_stats.apply(
    lambda r: f"[{r['bmi_q05']:.2f}, {r['bmi_q95']:.2f}]", axis=1
)
cluster_stats["bmi_range_min_max"] = cluster_stats.apply(
    lambda r: f"[{r['bmi_min']:.2f}, {r['bmi_max']:.2f}]", axis=1
)

# 输出：簇映射与区间表、带新簇标签的数据
cluster_stats[
    [
        "cluster",
        "cluster_sorted",
        "bmi_median",
        "bmi_range_q5_q95",
        "bmi_range_min_max",
        "n",
    ]
], features_km

(   cluster  cluster_sorted  bmi_median bmi_range_q5_q95 bmi_range_min_max    n
 0        0               0   29.808030   [28.07, 31.26]    [20.70, 31.56]  118
 1        3               1   30.252209   [28.17, 33.26]    [28.04, 34.06]   28
 2        4               2   33.212210   [31.73, 35.65]    [31.56, 36.36]   93
 3        2               3   33.874064   [31.66, 37.16]    [31.57, 37.20]   11
 4        1               4   38.567493   [36.02, 45.36]    [34.93, 46.88]   17,
        id        x1        x2  censored  sample_weight        BMI  t_proxy  \
 0    A001 -1.246572  2.205301         0            1.0  28.125000    20.14   
 1    A002  0.497095  0.030966         0            1.0  33.331832    13.86   
 2    A003 -0.370127 -0.266793         0            1.0  30.742188    13.00   
 3    A004 -1.073692 -0.959256         0            1.0  28.641243    11.00   
 4    A005 -0.633554 -0.512617         0            1.0  29.955556    12.29   
 ..    ...       ...       ...       ...     